# **Notebook 6 - Figures and Global Analyses**

---

## **Overview**

This notebook produces the **main figures and summary analyses** of the DiverCity study.
It combines the outputs of previous notebooks (including DiverCity computations, attractor metrics, and travel-time analyses) to generate the paper’s key visualizations and empirical findings.

The figures include:

* **Figure 1c–e**: Global DiverCity distribution and DiverCity vs. radius.
* **Figure 2c–d**: Relationship between DiverCity and mobility attractor features.
* **Figure 3a–b** and **Figure 3c**: DiverCity and travel-time responses to speed limit reductions.

These plots serve as a comprehensive summary of how road network structure and speed-limit interventions
shape potential route diversification in cities worldwide.

---

## **Related Figures and Notebooks**

For completeness, the remaining figures in the paper can be reproduced or derived from other notebooks:

| Figure          | Description                                                                                                                                | Notebook                                         |
| --------------- | ------------------------------------------------------------------------------------------------------------------------------------------ | ------------------------------------------------ |
| **Figure 1a–b** | Examples of low and high DiverCity trips with overlapping and distinct near-shortest routes.                                               | **Notebook 5a – Visualize Near-Shortest Routes** |
| **Figure 1e**   | Relationship between DiverCity and congestion (based on TomTom travel-time data). *Cannot be shared due to proprietary data restrictions.* | —                                                |
| **Figure 2a–b** | City-level DiverCity maps and spatial distribution of mobility attractors.                                                                 | **Notebook 5b – Maps of DiverCity**              |
| **Figure 4**    | Effect of speed limit reductions on route distribution in a specific city (e.g., Rome).                                                    | **Notebook 5a – Visualize Near-Shortest Routes** |
| **Figure 5**    | Controlled simulations showing DiverCity behavior in simplified grid networks.                                                             | **Notebook 4 – Simplified Model**                |

---

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
from matplotlib import pyplot as plt
from my_utils import create_folder_if_not_exists

from results_utils import (process_city,
    load_speed_scenarios, load_travel_times, load_network_measures,
    compute_divercity_gain, compute_travel_time_difference, load_attractor_dispersion,
    compute_city_median_divercity, compute_node_divercity_table, load_distance_node_attractors
)

from figures import (
    create_fig1c, create_fig1d,
    create_fig2c, create_fig2d,
    create_fig3a, create_fig3b, create_fig3c
)

## **1. Parameters**

In [ ]:
# ------------------------------------------------------------
# Parameters and Global Settings
# ------------------------------------------------------------

# List of cities included in the analysis
list_cities = ["milan"] 

# Experiment identifiers
exp_id = "exp_osm"

# DiverCity computation parameters
k = 5
p = 0.1
eps = 0.3

# Output directory for generated figures
figures_dir = "./Figures/"
create_folder_if_not_exists(figures_dir)

print(f"Figures will be saved in: {figures_dir}")

## **2. Load DiverCity Results**

This section loads the DiverCity results for all cities analyzed in the experiment.  


In [ ]:
dict_df_results = {}

for city in list_cities:
    city, df = process_city(city, f"../data/results/{city}_{exp_id}/", max_k=k)
    dict_df_results[city] = df

## **3. Figure 1 - Global DiverCity Patterns**

This section reproduces the main global patterns of DiverCity reported in the study.  
It includes two panels:

- **Figure 1c - DiverCity Distribution Across Cities:**  
  Shows the distribution of median DiverCity values across all analyzed cities.  

- **Figure 1d - DiverCity vs. Radius:**  
  Displays the average DiverCity as a function of distance from the city center, aggregated across all cities.  


**Figure 1c - DiverCity distribution across cities**

In [ ]:
fig, ax = create_fig1c(dict_df_results, list_cities, k=5, p=0.1, eps=0.3, cities_to_annotate=["rome", "milan"])

plt.tight_layout()
plt.savefig(f"{figures_dir}/Fig1c_DiverCity_Distribution.pdf", bbox_inches="tight")

**Figure 1d - DiverCity vs. Radius**  

In [ ]:
fig, ax = create_fig1d(dict_df_results, list_cities, k, p, eps)

plt.tight_layout()
plt.savefig(f"{figures_dir}/Fig1d_DiverCity_vs_Radius.pdf", bbox_inches="tight")

## **4. Figure 2 - Mobility Attractors and Spatial DiverCity Patterns**

This section analyzes how **mobility attractors** (e.g., highways, trunks, and ring roads)  
affect route diversification at both the node and city levels.  

It reproduces the results shown in *Figure 2* of the paper, composed of two main panels:

- **Figure 2c - Node-Level DiverCity vs. Distance to Attractors:**  
  Shows how DiverCity varies with the distance from mobility attractors across all cities.  

- **Figure 2d - City-Level DiverCity vs. Attractor Characteristics:**  
  Relates each city’s median DiverCity to the density and spatial dispersion of its attractors.  

In [ ]:
# ------------------------------------------------------------
# Compute city- and node-level metrics for attractor analysis
# ------------------------------------------------------------

# Compute per-city DiverCity medians and interquartile ranges
city_to_median_dc, city_to_iqr_dc = compute_city_median_divercity(
    dict_df_results,
    list_cities,
    k=k,
    p=p,
    eps=eps
)

# Load network- and attractor-related measures
city_to_attr_length = load_network_measures(list_cities)
city_to_attr_density, city_to_attr_H = load_attractor_dispersion(
    list_cities,
    city_to_attr_length=city_to_attr_length
)

# Load node-level distances to attractors
dict_distance_node_attractors = load_distance_node_attractors(list_cities)

# Build combined node-level DiverCity vs. distance table
df_node_dcity, list_percentiles = compute_node_divercity_table(
    list_cities,
    dict_df_results,
    dict_distance_node_attractors,
    p,
    eps,
    k
)

**Figure 2c - Node-Level DiverCity vs. Distance to Attractors**

In [ ]:
fig, ax = create_fig2c(df_node_dcity, list_percentiles, k, 
                       save=True,
                       savepath=f"{figures_dir}/Fig2c_Node_DiverCity_vs_Distance.pdf")

**Figure 2d - City-Level DiverCity vs. Attractor Characteristics**

In [ ]:
fig, ax = create_fig2d(city_to_attr_density, city_to_median_dc, city_to_attr_H, list_cities,
                       save=True,
                       savepath=f"{figures_dir}/Fig2d_City_DiverCity_vs_AttractorMetrics.pdf")

## **5. Figure 3 - Effects of Speed-Limit Tuning**

This section examines how **speed-limit reductions on mobility attractors** influence DiverCity and travel times.  
It reproduces *Figure 3* of the paper, consisting of two main panels:

- **Figure 3a - DiverCity Response to Speed Reductions:**  
  Shows how city-level DiverCity varies as the speed limits of attractors are progressively reduced.  


- **Figure 3b - Travel-Time Response to Speed Reductions:**  
  Quantifies the trade-off between increased route diversification and longer travel times.  

- **Figure 3c - City-Specific DiverCity Profile:**  
  DiverCity as a function of radial distance from the city center under different attractor speed scenarios.

In [ ]:
# Speed reduction scenarios (expressed as percentages of the original speed)
list_speeds = [10, 20, 30, 40, 50, 90]

# Experiment identifier
exp_id = "exp_osm"

# ------------------------------------------------------------
# Load data
# ------------------------------------------------------------

dict_df_speed = load_speed_scenarios(list_cities, list_speeds, exp_id, k)
dict_travel_times = load_travel_times(list_cities, p)

# Compute effects
dict_gains_DiverCity_speed = compute_divercity_gain(
    dict_df_results, dict_df_speed, list_cities, list_speeds, k, p, eps
)
dict_tt_difference = compute_travel_time_difference(
    dict_travel_times, list_cities, list_speeds
)

**Figure 3a - DiverCity Response to Speed Reductions**

In [ ]:
fig, ax = create_fig3a(dict_gains_DiverCity_speed, list_speeds, show_cities=["rome"],
                       save=True,
                       savepath=f"{figures_dir}/Fig3a_DiverCity_vs_SpeedReduction.pdf")

**Figure 3b - Travel-Time Response to Speed Reductions**

In [ ]:
fix, ax = create_fig3b(dict_tt_difference, list_speeds, show_cities=["milan"],
                       save=True,
                       savepath=f"{figures_dir}/Fig3b_TravelTime_vs_SpeedReduction.pdf")

**Figure 3c - City-Specific DiverCity Profile**

In [ ]:
city = "milan"

fix, ax = create_fig3c(city, dict_df_results, dict_df_speed, list_speeds, 
                       k, p, eps, save=True, savepath=f"{figures_dir}/Fig3c_{city}_speed_profiles.png")